In [1]:
%cd '/app'

/app


/home/user/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import os
import sys 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
sys.path.append('/app/src')

In [4]:
from loader.data import _load_data, SimpleDataCollator, HybridDataCollator
from loader.model import load_model

In [5]:
from dataset.tokernizer import set_tokenizer, set_vocab

In [7]:
num_variables = 2
field = 'QQ'
max_coefficient = 100
max_degree=20
continous_ids = [2]

params = {'encoding_method': 'hybrid',
          'd_model': 256,
          'nhead': 4,
          'num_encoder_layers': 2,
          'num_decoder_layers': 2,
          'dim_feedforward': 1024,
          'dropout': 0.1,
          'max_sequence_length': 512,
          'positional_encoding': 'embedding',
          'regression_weight': 1.0,}

import argparse

params = argparse.Namespace(**params)

vocab = set_vocab(num_variables, 
                  field=field, 
                  max_coeff=max_coefficient, 
                  max_degree=max_degree, 
                  continuous_coefficient=False, 
                continuous_exponent=False)
tokenizer = set_tokenizer(vocab)
model = load_model(params, vocab=vocab, tokenizer=tokenizer)


In [8]:
task = 'shape'
encoding = 'hybrid'
n, field = 2, 'GF7'
data_name = f'{task}_n={n}_field={field}'
data_path = f'data/{task}/{data_name}/data_{field}_n={n}.test.lex.infix'

testset     = _load_data(data_path)

In [9]:
dc = HybridDataCollator(tokenizer)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, collate_fn=dc, shuffle=False)

In [10]:
batch = next(iter(testloader))

In [46]:
# task = 'shape'
# encoding = 'standard'

# n, field = 2, 'GF7'
# data_name = f'{task}_n={n}_field={field}'
# data_path = f'data/{task}/{data_name}/data_{field}_n={n}.test.lex.infix'

# data_config_path = f'config/{data_name}.yaml'
# _save_path = f'{field}_n={n}_ep=8_bs=16'
# save_path = f'results/{task}/{encoding}_embedding/{_save_path}'
# # save_path = f'results/{task}/{encoding}/dryrun'

In [11]:
for key in batch:
    batch[key] = batch[key].to(model.device) if isinstance(batch[key], torch.Tensor) else batch[key]

In [12]:
model(**batch)

{'loss': tensor(19.9327, device='cuda:0', grad_fn=<AddBackward0>),
 'loss_clf': tensor(5.6199, device='cuda:0', grad_fn=<NllLossBackward0>),
 'loss_rg': tensor(14.3128, device='cuda:0', grad_fn=<MseLossBackward0>),
 'logits': tensor([[[-0.7997,  0.1777, -0.4062,  ...,  0.0946, -0.1072, -0.6497],
          [ 0.1215,  0.7225, -0.7747,  ...,  1.2960, -0.0859, -0.2309],
          [-0.5851, -0.0074, -0.6108,  ..., -0.1392,  0.0496, -1.0368],
          ...,
          [-0.2318,  0.3845, -1.1978,  ...,  0.1619, -0.4243,  1.0051],
          [-0.5707,  0.4581, -0.5036,  ...,  0.0650, -0.1025,  0.2354],
          [-0.5423, -0.5598, -0.8385,  ...,  0.3655,  1.3766, -0.0080]],
 
         [[-0.8181, -0.0689, -0.1057,  ..., -0.0082, -0.4242, -0.6536],
          [ 0.0495,  0.5423, -0.7644,  ...,  1.3736, -0.0865, -0.0540],
          [-0.6605,  0.0864, -0.4809,  ..., -0.0854,  0.1100, -1.1429],
          ...,
          [-0.3453,  0.0089, -1.0978,  ...,  0.1233, -0.3788,  0.5984],
          [-0.4874,  0

In [32]:
model.greedy_generate(batch['encoder_input'], 
                      encoder_input_labels=batch['encoder_input_labels'], 
                      max_length=500, 
                      encoder_padding_mask=batch['encoder_padding_mask'], 
                      continuous_token_ids=torch.tensor([tokenizer.vocab['[C]']]).to(model.device),
                      )

(tensor([[231, 136, 136,  ...,  26,  25,  10],
         [231,  54, 149,  ..., 170, 151,  89],
         [231,  54, 149,  ...,  10, 173,   5],
         [231,  54, 186,  ..., 172,  79,  91]], device='cuda:0'),
 tensor([[inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf]], device='cuda:0'))

In [33]:
num_variables = 2
field = 'QQ'
max_coefficient = 100
max_degree=20
continous_ids = [2]

params = {'encoding_method': 'standard',
          'd_model': 256,
          'nhead': 4,
          'num_encoder_layers': 2,
          'num_decoder_layers': 2,
          'dim_feedforward': 1024,
          'dropout': 0.1,
          'max_sequence_length': 512,
          'positional_encoding': 'embedding',
          'regression_weight': 1.0,}

import argparse

params = argparse.Namespace(**params)

vocab = set_vocab(num_variables, 
                  field=field, 
                  max_coeff=max_coefficient, 
                  max_degree=max_degree, 
                  continuous_coefficient=False, 
                continuous_exponent=False)
tokenizer = set_tokenizer(vocab)
model = load_model(params, vocab=vocab, tokenizer=tokenizer)


In [124]:
bag = load_pretrained_bag(save_path)
config, model, tokenizer = bag['config'], bag['model'], bag['tokenizer']

NameError: name 'load_pretrained_bag' is not defined

In [156]:
from loader.data import _load_data
from loader.data import SimpleDataCollator
from torch.utils.data import DataLoader
import numpy as np 
from tqdm import tqdm

batch_size = 500
dataloader = data_path
disable_tqdm = False
max_length = 10000

if isinstance(dataloader, str):
    dataset = _load_data(dataloader)
    dc = SimpleDataCollator(tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=dc, shuffle=False)

# load model    
if isinstance(model, str):
    bag = load_pretrained_bag(model)
    config, model, tokenizer = bag['config'], bag['model'], bag['tokenizer']
else:
    assert(tokenizer is not None)


model.cuda()

with torch.no_grad():
    hits = []
    dataloader = tqdm(dataloader, disable=disable_tqdm)  if not disable_tqdm else dataloader
    for batch in dataloader:
        max_length = min(max_length, batch['labels'].shape[1] + 1)
        outputs = model.greedy_generate(batch['encoder_input'].cuda(), max_length=max_length, encoder_padding_mask=batch['encoder_padding_mask'].cuda())
        pred = tokenizer.batch_decode(outputs.cpu().numpy(), skip_special_tokens=True)
        target = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
        
        hits += [p == t for p, t in zip(pred, target)]
        
    ret = {'acc': np.array(hits, dtype=float).mean(), 'hits': hits}

100%|██████████| 2/2 [00:10<00:00,  5.01s/it]


In [157]:
ret['acc']

0.432

: 

In [132]:
ret['hits'][:10]

[True, False, True, True, True, False, True, False, True, False]

In [133]:
batch.keys()

dict_keys(['encoder_input', 'decoder_input', 'encoder_padding_mask', 'decoder_padding_mask', 'labels'])

In [134]:
batch  = next(iter(dataloader))

In [149]:
%%time
outputs = model.greedy_generate(batch['encoder_input'].cuda(), 
                                encoder_attention_mask=None,
                                encoder_padding_mask=batch['encoder_padding_mask'].cuda(),
                                max_length=max_length)

CPU times: user 20.9 s, sys: 48.1 ms, total: 20.9 s
Wall time: 4.61 s


In [150]:
outputs[:4]

tensor([[36,  4, 12, 11, 32,  4, 11, 11, 40,  4, 11, 12, 38, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  4, 11, 13, 40,  4, 11, 14, 32,  5, 11, 13, 32,  5,
         11, 12, 32,  5, 11, 11, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 15, 38, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  8, 11, 14, 32,  9, 11, 12, 32,  9, 11, 11, 40,  4,
         11, 15, 32,  9, 11, 14, 32,  8, 11, 13, 32,  8, 11, 12, 38, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36]], device='cuda:0')

In [142]:
batch['labels'][:4]

tensor([[ 4, 12, 11, 32,  4, 11, 11, 40,  4, 11, 12, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  4, 11, 13, 40,  4, 11, 14, 32,  8, 11, 11, 38, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 15, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  8, 11, 14, 32,  9, 11, 12, 32,  9, 11, 11, 40,  4, 11,
         15, 32,  9, 11, 14, 32,  8, 11, 13, 32,  8, 11, 12, 38, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36]])

In [151]:
tokenizer.batch_decode(outputs.cpu().numpy(), skip_special_tokens=True)[1]

'C1 E1 E0 + C1 E0 E2 [SEP] C1 E0 E3 + C2 E0 E2 + C2 E0 E1 + C2 E0 E0'

In [147]:
tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)[1]

'C1 E1 E0 + C1 E0 E2 [SEP] C1 E0 E3 + C5 E0 E0'

In [148]:
tokenizer.batch_decode(batch['encoder_input'], skip_special_tokens=True)[1]


'C1 E1 E0 + C1 E0 E2 [SEP] C1 E3 E1 + C1 E2 E3 + C5 E2 E2 + C5 E1 E4 + C1 E1 E2 + C5 E1 E1 + C3 E1 E0 + C6 E0 E3 + C3 E0 E2 + C2 E0 E1 + C5 E0 E0 [SEP] C4 E5 E2 + C4 E4 E4 + C4 E4 E1 + C4 E3 E3 + C6 E3 E2 + C5 E3 E1 + C3 E2 E5 + C3 E2 E4 + C5 E2 E3 + C3 E2 E1 + C1 E2 E0 + C2 E1 E4 + C1 E1 E3 + C1 E1 E2 + C2 E0 E4 + C5 E0 E3 + C1 E0 E2 + C3 E0 E1 + C2 E0 E0'

In [71]:
tokenizer.vocab

{'E9': 20,
 'C0': 3,
 'E12': 23,
 '[UNK]': 41,
 '/': 35,
 'E10': 21,
 'E2': 13,
 'C2': 5,
 'E17': 28,
 '[SEP]': 40,
 '^': 34,
 'E14': 25,
 'E18': 29,
 '</s>': 38,
 '<s>': 37,
 'E3': 14,
 '+': 32,
 'C3': 6,
 'E8': 19,
 'E13': 24,
 'E0': 11,
 'E5': 16,
 '[PAD]': 36,
 'E4': 15,
 'C4': 7,
 'E20': 31,
 'C1': 4,
 'C5': 8,
 '[C]': 2,
 'C6': 9,
 '[E]': 10,
 'E19': 30,
 '[CLS]': 39,
 '*': 33,
 'E16': 27,
 'E1': 12,
 'E11': 22,
 'x0': 0,
 'x1': 1,
 'E15': 26,
 'E7': 18,
 'E6': 17}